<a href="https://colab.research.google.com/github/Tasnia16EEE/Twitter-Sentiment-Analysis-/blob/main/bert_multilabel_classification_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!nvidia-smi

Thu Jul 31 21:48:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             32W /   70W |    2462MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [44]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys
import re

In [45]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/twitter-entity-sentiment-analysis


In [47]:
import os

# List files in the dataset folder
os.listdir(path)


['twitter_validation.csv', 'twitter_training.csv']

In [48]:
import pandas as pd

# Example: if the file is named 'train.csv'
df = pd.read_csv(os.path.join(path, "twitter_training.csv"), header=None)
df.columns = ["tweet_id", "entity", "sentiment", "text"]

# Preview the dataset
df.head()


,tweet_id,entity,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [49]:
df["sentiment"].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [50]:
# One-hot encode the sentiment column
one_hot = pd.get_dummies(df["sentiment"])

# Add the new columns to the original DataFrame
df = pd.concat([df, one_hot], axis=1)

df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,False,False,False,True
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,False,False,False,True
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,False,False,False,True
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,False,False,False,True
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,False,False,False,True


In [51]:
df[["Positive", "Negative", "Neutral", "Irrelevant"]] = df[["Positive", "Negative", "Neutral", "Irrelevant"]].astype(int)
df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,0,0,0,1
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,0,0,0,1
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,0,0,0,1
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,0,0,0,1
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,0,0,0,1


In [52]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+",'',text)
    text = re.sub(r"@\w+",'',text)
    text = re.sub(r"#\w+",'',text)
    text = re.sub(r"[^a-zA-Z\s]",'',text)
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [53]:
df["text"] = df["text"].apply(clean_text)
df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,2401,Borderlands,Positive,im getting borderlands murder,0,0,0,1
1,2401,Borderlands,Positive,coming borders kill,0,0,0,1
2,2401,Borderlands,Positive,im getting borderlands kill,0,0,0,1
3,2401,Borderlands,Positive,im coming borderlands murder,0,0,0,1
4,2401,Borderlands,Positive,im getting borderlands murder,0,0,0,1


In [54]:
df["CONTEXT"] = df["entity"] + ". " + df["text"]
df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive,CONTEXT
0,2401,Borderlands,Positive,im getting borderlands murder,0,0,0,1,Borderlands. im getting borderlands murder
1,2401,Borderlands,Positive,coming borders kill,0,0,0,1,Borderlands. coming borders kill
2,2401,Borderlands,Positive,im getting borderlands kill,0,0,0,1,Borderlands. im getting borderlands kill
3,2401,Borderlands,Positive,im coming borderlands murder,0,0,0,1,Borderlands. im coming borderlands murder
4,2401,Borderlands,Positive,im getting borderlands murder,0,0,0,1,Borderlands. im getting borderlands murder


In [55]:
train_df = df.copy()


In [56]:
# dropping useless features/columns
train_df.drop(labels=['tweet_id', 'entity', 'sentiment'], axis=1, inplace=True)

In [57]:
train_df.head()

,text,Irrelevant,Negative,Neutral,Positive,CONTEXT
0,im getting borderlands murder,0,0,0,1,Borderlands. im getting borderlands murder
1,coming borders kill,0,0,0,1,Borderlands. coming borders kill
2,im getting borderlands kill,0,0,0,1,Borderlands. im getting borderlands kill
3,im coming borderlands murder,0,0,0,1,Borderlands. im coming borderlands murder
4,im getting borderlands murder,0,0,0,1,Borderlands. im getting borderlands murder


In [58]:
# dropping useless features/columns
train_df.drop(labels=['text'], axis=1, inplace=True)

In [59]:
train_df.head()

,Irrelevant,Negative,Neutral,Positive,CONTEXT
0,0,0,0,1,Borderlands. im getting borderlands murder
1,0,0,0,1,Borderlands. coming borders kill
2,0,0,0,1,Borderlands. im getting borderlands kill
3,0,0,0,1,Borderlands. im coming borderlands murder
4,0,0,0,1,Borderlands. im getting borderlands murder


In [60]:
# rearranging columns
train_df = train_df[['CONTEXT', 'Positive', 'Negative', 'Neutral', 'Irrelevant']]
train_df.head()

,CONTEXT,Positive,Negative,Neutral,Irrelevant
0,Borderlands. im getting borderlands murder,1,0,0,0
1,Borderlands. coming borders kill,1,0,0,0
2,Borderlands. im getting borderlands kill,1,0,0,0
3,Borderlands. im coming borderlands murder,1,0,0,0
4,Borderlands. im getting borderlands murder,1,0,0,0


In [61]:
train_df

,CONTEXT,Positive,Negative,Neutral,Irrelevant
0,Borderlands. im getting borderlands murder,1,0,0,0
1,Borderlands. coming borders kill,1,0,0,0
2,Borderlands. im getting borderlands kill,1,0,0,0
3,Borderlands. im coming borderlands murder,1,0,0,0
4,Borderlands. im getting borderlands murder,1,0,0,0
...,...,...,...,...,...
74677,Nvidia. realized windows partition mac like ye...,1,0,0,0
74678,Nvidia. realized mac window partition years be...,1,0,0,0
74679,Nvidia. realized windows partition mac years b...,1,0,0,0
74680,Nvidia. realized windows partition mac like ye...,1,0,0,0


In [62]:
# Get 30 samples from each class
positive_df = train_df[train_df['Positive'] == 1].sample(n=50, random_state=42)
negative_df = train_df[train_df['Negative'] == 1].sample(n=50, random_state=42)
neutral_df  = train_df[train_df['Neutral']  == 1].sample(n=50, random_state=42)
irrelevant_df = train_df[train_df['Irrelevant'] == 1].sample(n=50, random_state=42)

# Combine into one dataset
balanced_df = pd.concat([positive_df, negative_df, neutral_df, irrelevant_df], ignore_index=True)

# Shuffle the final dataset (optional)
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [63]:
train_df = balanced_df.copy()

In [64]:
train_df

,CONTEXT,Positive,Negative,Neutral,Irrelevant
0,GrandTheftAuto(GTA). man put support ticket w ...,0,1,0,0
1,CallOfDutyBlackopsColdWar. love see,1,0,0,0
2,HomeDepot. thank ehn members employed stores p...,1,0,0,0
3,FIFA. immediately loved till fernando lost dio...,0,0,0,1
4,Borderlands. today uploaded ton vods youtube a...,0,0,1,0
...,...,...,...,...,...
195,Google. would succeed join provide google camp...,0,0,1,0
196,RedDeadRedemption(RDR). play friend red dead r...,1,0,0,0
197,GrandTheftAuto(GTA). unk mad fuck next gta chi...,0,1,0,0
198,CallOfDuty. come live mixer play new warzone b...,0,0,0,1


## Processing Validation Data

In [65]:
val_df = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv", header=None)
val_df.columns = ["tweet_id", "entity", "sentiment", "text"]
val_df.head()

,tweet_id,entity,sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [66]:
val_df["sentiment"].unique()

array(['Irrelevant', 'Neutral', 'Negative', 'Positive'], dtype=object)

In [67]:
# One-hot encode the sentiment column
one_hot = pd.get_dummies(val_df["sentiment"])

# Add the new columns to the original DataFrame
val_df = pd.concat([val_df, one_hot], axis=1)

val_df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...,True,False,False,False
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,False,False,True,False
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,False,True,False,False
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",False,True,False,False
4,4433,Google,Neutral,Now the President is slapping Americans in the...,False,False,True,False


In [68]:
val_df[["Positive", "Negative", "Neutral", "Irrelevant"]] = val_df[["Positive", "Negative", "Neutral", "Irrelevant"]].astype("Int64")
val_df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...,1,0,0,0
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,0,0,1,0
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,0,1,0,0
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",0,1,0,0
4,4433,Google,Neutral,Now the President is slapping Americans in the...,0,0,1,0


In [69]:
val_df["text"] = val_df["text"].apply(clean_text)
val_df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive
0,3364,Facebook,Irrelevant,mentioned facebook struggling motivation go ru...,1,0,0,0
1,352,Amazon,Neutral,bbc news amazon boss jeff bezos rejects claims...,0,0,1,0
2,8312,Microsoft,Negative,pay word functions poorly chromebook,0,1,0,0
3,4371,CS-GO,Negative,csgo matchmaking full closet hacking truly awf...,0,1,0,0
4,4433,Google,Neutral,president slapping americans face really commi...,0,0,1,0


In [70]:
val_df["CONTEXT"] = val_df["entity"] + ". " + val_df["text"]
val_df.head()

,tweet_id,entity,sentiment,text,Irrelevant,Negative,Neutral,Positive,CONTEXT
0,3364,Facebook,Irrelevant,mentioned facebook struggling motivation go ru...,1,0,0,0,Facebook. mentioned facebook struggling motiva...
1,352,Amazon,Neutral,bbc news amazon boss jeff bezos rejects claims...,0,0,1,0,Amazon. bbc news amazon boss jeff bezos reject...
2,8312,Microsoft,Negative,pay word functions poorly chromebook,0,1,0,0,Microsoft. pay word functions poorly chromebook
3,4371,CS-GO,Negative,csgo matchmaking full closet hacking truly awf...,0,1,0,0,CS-GO. csgo matchmaking full closet hacking tr...
4,4433,Google,Neutral,president slapping americans face really commi...,0,0,1,0,Google. president slapping americans face real...


In [71]:
# dropping useless features/columns
val_df.drop(labels=['tweet_id', 'entity', 'sentiment', 'text'], axis=1, inplace=True)

In [72]:
# rearranging columns
val_df = train_df[['CONTEXT', 'Positive', 'Negative', 'Neutral', 'Irrelevant']]
val_df.head()

,CONTEXT,Positive,Negative,Neutral,Irrelevant
0,GrandTheftAuto(GTA). man put support ticket w ...,0,1,0,0
1,CallOfDutyBlackopsColdWar. love see,1,0,0,0
2,HomeDepot. thank ehn members employed stores p...,1,0,0,0
3,FIFA. immediately loved till fernando lost dio...,0,0,0,1
4,Borderlands. today uploaded ton vods youtube a...,0,0,1,0


In [73]:
# Get 30 samples from each class
positive_df = val_df[val_df['Positive'] == 1].sample(n=50, random_state=42)
negative_df = val_df[val_df['Negative'] == 1].sample(n=50, random_state=42)
neutral_df  = val_df[val_df['Neutral']  == 1].sample(n=50, random_state=42)
irrelevant_df = val_df[val_df['Irrelevant'] == 1].sample(n=50, random_state=42)

# Combine into one dataset
balanced_df = pd.concat([positive_df, negative_df, neutral_df, irrelevant_df], ignore_index=True)

# Shuffle the final dataset (optional)
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [74]:
val_df = balanced_df.copy()

In [75]:
val_df

,CONTEXT,Positive,Negative,Neutral,Irrelevant
0,Fortnite. fortnite must forgotten console new ...,0,1,0,0
1,Dota2. nice time finally chill good chill stream,1,0,0,0
2,AssassinsCreed. bored want see good story acti...,1,0,0,0
3,Nvidia. whew zoom onto future,0,0,0,1
4,CallOfDutyBlackopsColdWar. really hope black o...,0,0,1,0
...,...,...,...,...,...
195,TomClancysRainbowSix. hackers check c,0,0,1,0
196,Microsoft. wait still allow floppy disks thoug...,1,0,0,0
197,FIFA. say un fifa institution comes un cartel ...,0,1,0,0
198,ApexLegends. everybody sweaty today like fucki...,0,0,0,1


In [76]:
target_list = ['Positive', 'Negative', 'Neutral', 'Irrelevant']

In [77]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05

In [78]:
from transformers import BertTokenizer, BertModel

In [79]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [80]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['CONTEXT']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [81]:
train_size = 0.8

In [82]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [83]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [84]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [85]:
device

device(type='cuda')

In [86]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4) # Changed output size to 4

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [87]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [88]:
val_targets=[]
val_outputs=[]

In [89]:
import torch
import numpy as np

def train_model(n_epochs, training_loader, validation_loader, model, optimizer):
    valid_loss_min = np.inf

    for epoch in range(1, n_epochs + 1):
        train_loss = 0
        valid_loss = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += (1 / (batch_idx + 1)) * (loss.item() - train_loss)

        print('############# Epoch {}: Training End     #############'.format(epoch))
        print('############# Epoch {}: Validation Start   #############'.format(epoch))

        model.eval()
        val_targets = []
        val_outputs = []

        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader):
                ids = data['input_ids'].to(device, dtype=torch.long)
                mask = data['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.float)

                outputs = model(ids, mask, token_type_ids)
                loss = loss_fn(outputs, targets)

                valid_loss += (1 / (batch_idx + 1)) * (loss.item() - valid_loss)
                val_targets.extend(targets.cpu().detach().numpy().tolist())
                val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        print('############# Epoch {}: Validation End     #############'.format(epoch))

        train_loss /= len(training_loader)
        valid_loss /= len(validation_loader)

        print('Epoch: {} \tAverage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))

        # Inform about improvement (optional)
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(valid_loss_min, valid_loss))
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done')


In [90]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer)

############# Epoch 1: Training Start   #############


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Average Training Loss: 0.006043 	Average Validation Loss: 0.005566
Validation loss decreased (inf --> 0.005566).
############# Epoch {}  Done


In [91]:
# testing
example = "He was trying hard to get this, but couldn't"
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

Irrelevant


/tmp/ipython-input-3243459557.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])


In [93]:
# testing
example = "he is bad"
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.26635605096817017, 0.36006879806518555, 0.3288143575191498, 0.3699794411659241]]
Irrelevant


/tmp/ipython-input-2674994758.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])


In [94]:
from sklearn.metrics import classification_report
import numpy as np

model.eval()
val_targets = []
val_outputs = []

with torch.no_grad():
    for data in val_data_loader:
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].cpu().numpy()  # One-hot format
        outputs = model(ids, mask, token_type_ids)
        probs = torch.sigmoid(outputs).cpu().numpy()

        val_targets.extend(targets)
        val_outputs.extend(probs)


In [95]:
# Convert one-hot true labels to class indices
y_true = np.argmax(val_targets, axis=1)

# Convert predicted probabilities to class indices
y_pred = np.argmax(val_outputs, axis=1)


In [ ]:
from sklearn.metrics import classification_report

target_names = ['Positive', 'Negative', 'Neutral', 'Irrelevant']

print(classification_report(y_true, y_pred, target_names=target_names))
